In [ ]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder

In [ ]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train.drop([
    'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
    'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
    'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
    ], axis=1)
X_test = X_test.drop([
    'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
    'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
    'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
    ], axis=1)

In [ ]:
cat_features = [
    'Поставщик', 'Материал', 'Операционный менеджер',
    'Завод', 'Закупочная организация', 'Группа закупок', 'Балансовая единица',
    'ЕИ', 'Вариант поставки'
    ]

In [ ]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.bin_encoder = BinaryEncoder(cols=cat_features)

    def fit(self, X, y=None):
        self.bin_encoder.fit(X)
        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        return X_


In [19]:
# data_preprocessor.fit(X_train)
# X_train_preproc = data_preprocessor.transform(X_train)
# X_test_preproc = data_preprocessor.transform(X_test)

In [24]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 1000, 3000, 50)
    max_depth = trial.suggest_int('max_depth', 6, 24)
    max_bin = trial.suggest_int('max_bin', 64, 256),
    num_leaves = trial.suggest_int('num_leaves', 64, 512)
    reg_lambda = trial.suggest_float('l2_reg', 0.1, 1.5, log=True)

    # Модель
    data_preprocessor = DataPreprocessor(cat_features)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        n_jobs=-1,
        force_col_wise=True,
        verbose=-1
    )

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)
    accuracy = f1_score(y_test, pipeline.predict(X_test), average='macro')
    
    # cv_score = cross_val_score(model, X_train_lgbm, y_train, cv=StratifiedKFold(), scoring='f1_macro', n_jobs=-1)
    # accuracy = cv_score.mean()

    return accuracy

In [ ]:
X_train_lgbm

In [25]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2023-08-07 23:29:13,649] A new study created in memory with name: no-name-47d449c8-1f2f-49bf-bb9c-81b806f34e47
[I 2023-08-07 23:29:53,500] Trial 0 finished with value: 0.8789539350045279 and parameters: {'learning_rate': 0.19388718090331294, 'n_estimators': 1450, 'max_depth': 21, 'max_bin': 246, 'num_leaves': 249, 'l2_reg': 0.823308345599237}. Best is trial 0 with value: 0.8789539350045279.
[I 2023-08-07 23:30:30,023] Trial 1 finished with value: 0.8824268858847433 and parameters: {'learning_rate': 0.3347896437275221, 'n_estimators': 1400, 'max_depth': 16, 'max_bin': 229, 'num_leaves': 449, 'l2_reg': 0.93121791929494}. Best is trial 1 with value: 0.8824268858847433.
[I 2023-08-07 23:31:06,506] Trial 2 finished with value: 0.88227888256637 and parameters: {'learning_rate': 0.27196959653600294, 'n_estimators': 1500, 'max_depth': 21, 'max_bin': 207, 'num_leaves': 358, 'l2_reg': 0.5938952437260764}. Best is trial 1 with value: 0.8824268858847433.
[I 2023-08-07 23:32:03,246] Trial 3 fini

KeyboardInterrupt: 

In [ ]:
study.best_params